<a href="https://colab.research.google.com/github/IgnacioGeo/Deliverables/blob/main/ProjectImages2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.backend import clear_session

# Clear any previous models from memory
clear_session()

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the images
x_train_normalized = x_train.astype('float32') / 255.0
x_test_normalized = x_test.astype('float32') / 255.0

# Convert labels to one-hot encoded form
y_train_one_hot = to_categorical(y_train, 10)
y_test_one_hot = to_categorical(y_test, 10)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,   # Randomly rotate images
    width_shift_range=0.1,  # Randomly shift images horizontally
    height_shift_range=0.1,  # Randomly shift images vertically
    horizontal_flip=True  # Randomly flip images horizontally
)

# Fit the data generator to the training data
datagen.fit(x_train_normalized)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [2]:
#Model Architecture
# Create a new Sequential model
model = Sequential()

# Add convolutional and pooling layers
model.add(Conv2D(128, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten the output
model.add(Flatten())

# Add dense layers with dropout
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# Print the model summary
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 30, 30, 128)         │           3,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 15, 15, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 13, 13, 64)          │          73,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 6, 6, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 4, 4, 32)            │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 2, 2, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │          66,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           5,130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 167,018 (652.41 KB)

 Trainable params: 167,018 (652.41 KB)

 Non-trainable params: 0 (0.00 B)

In [3]:
# Model Training
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Fit the training data with data augmentation
history = model.fit(datagen.flow(x_train_normalized, y_train_one_hot, batch_size=64),
                    epochs=100, validation_data=(x_test_normalized, y_test_one_hot),
                    callbacks=[early_stopping])


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


782/782 ━━━━━━━━━━━━━━━━━━━━ 235s 297ms/step - accuracy: 0.2619 - loss: 1.9615 - val_accuracy: 0.4587 - val_loss: 1.4707
Epoch 2/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 256s 290ms/step - accuracy: 0.4558 - loss: 1.4792 - val_accuracy: 0.5257 - val_loss: 1.3015
Epoch 3/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 226s 289ms/step - accuracy: 0.5152 - loss: 1.3466 - val_accuracy: 0.5791 - val_loss: 1.1932
Epoch 4/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 226s 289ms/step - accuracy: 0.5483 - loss: 1.2677 - val_accuracy: 0.6013 - val_loss: 1.1082
Epoch 5/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 226s 289ms/step - accuracy: 0.5612 - loss: 1.2242 - val_accuracy: 0.6090 - val_loss: 1.1116
Epoch 6/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 225s 288ms/step - accuracy: 0.5871 - loss: 1.1659 - val_accuracy: 0.6189 - val_loss: 1.0791
Epoch 7/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 224s 287ms/step - accuracy: 0.5897 - loss: 1.1495 - val_accuracy: 0.6118 - val_loss: 1.1059
Epoch 8/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 226s 288ms/step - accuracy: 0.6047 - los

In [4]:
# Save the trained model
model.save('cifar10_augmented_cnn_model.keras')
print('Model saved as cifar10_augmented_cnn_model.keras')

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(x_test_normalized, y_test_one_hot)
print(f'Test Accuracy with Data Augmentation: {test_accuracy}')

Model saved as cifar10_augmented_cnn_model.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 13s 40ms/step - accuracy: 0.7233 - loss: 0.7963
Test Accuracy with Data Augmentation: 0.7242000102996826
